Sequence <a href="https://oeis.org/A130229">A130229</a> in the OEIS consists of primes $p \equiv 5 \bmod 8$ for which the Pellian equation 
$$
x^2 - p y^2 = -4
$$
no solutions in odd integers. Equivalently, let $K = \mathbb{Q}(\sqrt{p})$ with ring of integers $\mathcal{O}_K = \mathbb{Z}[\frac{1+\sqrt{p}}{2}]$ and fundamental unit $\varepsilon_p = \frac{x_0 + y_0\sqrt{p}}{2}$. Here $(x_0, y_0)$ is a fundamental solution to the Pellian equation above and $p$ is an element of A130229 if and only if $x_0$ and $y_0$ are even, equivalently $\varepsilon_p \equiv 1 \bmod 2\mathcal{O}_K$. 

Note that $2$ is inert in $K/\mathbb{Q}$, so there are three possible non-zero residue classes for $\varepsilon_p \bmod 2\mathcal{O}_K$. Heuristically, one thus expects about one third of all primes $p \equiv 5 \bmod 8$ to be members of this sequence.

The present notebook contains code to test this heuristic. We define two counting functions for the sequence A130229. The naive counting function is
$$\pi_1(x) = \sum_{p\in A130229, \; p\leq x} 1.$$
For a better counting function, first define
$$\chi(p) = \left\{ \begin{array}{ll} 
1 & \text{if $p \equiv 5 \bmod 8$ and $\varepsilon_p \equiv 1 \bmod 2\mathcal{O}_K$} \\
-\frac{1}{2} & \text{if $p \equiv 5 \bmod 8$ and $\varepsilon_p \not\equiv 1 \bmod 2\mathcal{O}_K$} \\
0 & \text{if $p \not\equiv 5 \bmod 8$.}
\end{array}
\right.
$$
Now set 
$$
\theta_\chi(x) := \sum_{p \leq x} \chi(p)\log p.
$$
Our heuristc leads us to expect that $\theta_\chi(x) = o(x)$, i.e. that the terms mostly cancel out.

The code below computes the fundamental solution $(x_0, y_0) \bmod 2$ using the continued fraction method, with $B_i$ and $G_i$ reduced mod 2 at each step. The code computes this in parallel for many primes $p$ on a GPU. 



In [1]:
from sympy import isprime
from datetime import datetime
import math
import numpy as np
from time import localtime, strftime   
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
from numba import cuda


# first, build a sieving list of primes below 2**20:
small_primes = []

# Yes, it's inefficient, but we only run this once, so that makes no difference
for p in range(3, 2**20):
    if isprime(p):
        small_primes.append(p)

small_primes_arr = np.array(small_primes, dtype=np.uint32)


def size_str(n):
    '''Return str describing size of n bytes in kB, MB, GB,...'''
    if n < 1024:
        return f'{n} bytes'
    elif n < 1048576:
        return f'{(n/1024):.04} kB'
    elif n < 1073741824:
        return f'{(n/1048576):.04} MB'
    elif n < 1099511627776:
        return f'{(n/1073741824):.04} GB'
    else:
        return f'{(n/1099511627776):.04} TB'


# This is the Python function you want to run on the GPU
@cuda.jit
def Pell_Lambda_sieved_kernel(input_array, output_array): #, error_flag):
    '''
    This is supposed to run on a GPU.
    input_array contains prime numbers congruent to 5 mod 8.
    output_array will store the outputs of the form log(p) or -0.5*log(p), depending on the parity of the fundamental unit.
    '''
    idx = cuda.grid(1)
    if idx < input_array.size:  # Check array bounds
        p = input_array[idx]
        sqrt_p = np.uint64(math.sqrt(np.float64(p))) 
        P = np.uint64(1)
        Q = np.uint64(2)
        q = np.uint64((1 + sqrt_p) / 2)
        B = np.uint8(1)
        B_old = np.uint8(0)
        G = np.uint8(((2*q) - 1) % 2)
        G_old = np.uint8(0)
    
        # First iteration
        P = (q*Q) - P
        Q = (p - (P*P)) / Q
        q = np.uint64((P + sqrt_p) / Q)
        # G_old, G = G, ((q*G + G_old) % 2)
        # B_old, B = B, ((q*B + B_old) % 2)
        G_old, G = G, (q & G & 1) ^ G_old
        B_old, B = B, (q & B & 1) ^ B_old
    
        # main loop:
        while Q != 2:
            P = (q*Q) - P
            Q = (p - (P*P)) / Q
            q = np.uint64((P + sqrt_p) / Q)
            G_old, G = G, (q & G & 1) ^ G_old
            B_old, B = B, (q & B & 1) ^ B_old
            # G_old, G = G, ((q*G + G_old) % 2)
            # B_old, B = B, ((q*B + B_old) % 2)

            # # how big do P and Q get?
            # if P > error_flag[0]:
            #     error_flag[0] = P
            # if Q > error_flag[1]:
            #     error_flag[1] = Q

            # # check for potential overflow errors:
            # if P > 2147483648 or Q > 2147483648:
            #     error_flag[0] = 1

        if G_old == 0:
            output_array[idx] = math.log(np.float64(p))
        else:
            output_array[idx] = -0.5*math.log(np.float64(p))




def primes_5mod_8(a, b):
    '''Return array containing all primes 5 mod 8 in interval [a, b)'''
    if a < 5:
        N_1 = 0
    else:
        N_1 = np.int64(np.ceil((a - 5) / 8))
    if b < 5:
        N_2 = 0
    else:
        N_2 = np.int64(np.floor((b - 5) / 8))
        if 8*N_2 + 5 < b:
            N_2 += 1
    chunk_size = np.int64(N_2 - N_1)
    # print(N_1, N_2)
    # print(8*N_1+5, 8*N_2+5)
    if chunk_size <= 0:
        return np.array(0, dtype=np.uint64)
      
    sieve = np.ones(chunk_size, dtype=bool)  # initialise sieve
    
    for p in small_primes_arr:
        if np.int64(p)**2 >= b:  # only check primes < sqrt(N_8) - avoid overflow in squaring!
            break
        # get starting index k for 8(N + k) + 5 is a multiple of p
        d = (p & 7) ^ 4  # so that d*p = 5 mod 8
        k_start = np.int64(((d*p-5)//8 - N_1) % p)
        if 8*(k_start + N_1) + 5 == p:  # don't kill p itself
            k_start += p
        # now sieve:
        sieve[k_start :: p] = False

    return 8*(N_1 + np.nonzero(sieve)[0])+5


def theta_pi_kernel(a, b, verbose=True):
    '''Return theta(b) - theta(a)'''
    # first, create an array of prime numbers 5 mod 8 in the interval (a, b]
    if verbose:
        print(f'   Sieving primes in interval [{a+1}, {b+1})...')
    input_array = primes_5mod_8(a+1, b+1)
    if input_array.shape == () or input_array.size == 0:
        return 0.0, 0
    if verbose:
        print(f'    {input_array.size} primes found. Sending block [{input_array[0]}, ... ,{input_array[-1]}] of size {size_str(input_array.nbytes)} to GPU.')
    output_array = np.zeros_like(input_array, dtype=np.float64)  # for results
    d_input_array = cuda.to_device(input_array)
    d_output_array = cuda.to_device(output_array)

    # Run the kernel on the GPU
    threads_per_block = 256 # seems a good choice for my entry-level gaming laptop...
    blocks = (input_array.size + (threads_per_block - 1)) // threads_per_block
    Pell_Lambda_sieved_kernel[blocks, threads_per_block](d_input_array, d_output_array) #, d_error_flag)

    # Copy the results back to the host
    output_array = d_output_array.copy_to_host()
    theta_result = np.sum(output_array)
    pi_result = np.count_nonzero(output_array > 0)   # I hope this is right, we want to count the number of primes for which positive result is returned
    if verbose:
        print(f'     Result: theta =  {theta_result};  pi = {pi_result}')
    return theta_result, pi_result
    
    
    
def complete_theta(X, verbose=True):
    '''Return list of points (x, theta(x)) for x <= X.'''
    # first, create an array of prime numbers 5 mod 8 in the interval [a, b]
    if verbose:
        print(f'Sieving primes in interval [2, {X}]...')
    input_array = primes_5mod_8(2, X+1)
    if verbose:
        print(f' {input_array.size} primes found. Sending block of size {size_str(input_array.nbytes)} to GPU.')
    output_array = np.zeros_like(input_array, dtype=np.float64)  # for results
    d_input_array = cuda.to_device(input_array)
    d_output_array = cuda.to_device(output_array)

    # Run the kernel on the GPU
    threads_per_block = 256 #256
    blocks = (input_array.size + (threads_per_block - 1)) // threads_per_block
    Pell_Lambda_sieved_kernel[blocks, threads_per_block](d_input_array, d_output_array) #, d_error_flag)

    # Copy the results back to the host
    output_array = d_output_array.copy_to_host()
    if verbose:
        print(np.sum(output_array))

    # now read through the results and add up theta as we go
    result = []
    theta = 0.0
    i = 0
    for x in range(X+1):
        p = input_array[i]
        if x == p:
            theta += output_array[i]
            if i < input_array.size - 1:
                i += 1
        result.append((x, theta))
    return result


def A130229(X, verbose=True):
    '''
    Returns a list of members of sequence A130229 less than X.
    '''
    # first, create an array of prime numbers 5 mod 8 in the interval [a, b]
    if verbose:
        print(f'Sieving primes in interval [2, {X}]...')
    input_array = primes_5mod_8(2, X+1)
    if verbose:
        print(f' {input_array.size} primes found. Sending block of size {size_str(input_array.nbytes)} to GPU.')
    output_array = np.zeros_like(input_array, dtype=np.float64)  # for results
    d_input_array = cuda.to_device(input_array)
    d_output_array = cuda.to_device(output_array)

    # Run the kernel on the GPU
    threads_per_block = 256 #256
    blocks = (input_array.size + (threads_per_block - 1)) // threads_per_block
    Pell_Lambda_sieved_kernel[blocks, threads_per_block](d_input_array, d_output_array) #, d_error_flag)

    # Copy the results back to the host
    output_array = d_output_array.copy_to_host()
    if verbose:
        print(np.sum(output_array))

    # now return the results, i.e. primes p for which lambda(p) > 0
    result = []
    for p, l in zip(input_array, output_array):
        if l > 0:
            result.append(p)

    return result

In [13]:
# lets compute all members of sequence A130229 below 10^8 and write them to a file

sequence_filename = 'A130229.txt'
X = 10**8

with open(sequence_filename, 'w') as sequence_file:
    for p in A130229(X):
        sequence_file.write(str(p)+',\n')

Sieving primes in interval [2, 100000000]...
 1440534 primes found. Sending block of size 10.99 MB to GPU.
-306598.8227363084


What we really want is $\theta_\chi(x)$ for $x \leq 10^{11}$. That's too many points to store them all, so we'll compute a list of the form $(x, \theta_\chi(x))$, where $x$ runs through multiples of $10^n$ for suitable $n$.

In [4]:
# these are the main functions to call:

# don't make this too big - otherwise the GPU will be busy for 2h and we can't interrupt it...
MAX_INPUT_RANGE = 100000000   # widest range of inputs we'll send to GPU in one go

log_file_name = 'Pell_computations.log'

import logging
from datetime import datetime

# Configure the logging system
logging.basicConfig(
    filename=log_file_name,  # Log to this file
    level=logging.INFO,     # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s'  # Include timestamp
)

def theta_GPU_linear(x_start=0, dx=1, N_points=1000, save_file=None, pi_file=None, resume=False,  more_points=0, verbose=True):
    '''Compute Pell_theta function on a list of N_points spaced linearly dx apart and starting with X_start.'''
    logging.info(f'Launching theta_GPU_linear: x_start = {x_start}, dx = {dx}, N_points = {N_points}, save_file: {save_file}, pi_file: {pi_file}, resume: {resume}, more_points: {more_points}')
    if not resume:   # create new files, write headers
        if save_file is not None:
            with open(save_file, "a") as f:
                f.write('\n'+strftime('%c')+'\n'+'x_start = '+str(x_start)+', dx = '+str(dx)+'\n'+'N_points = '+str(N_points)+'\n')
        if pi_file is not None:
            with open(pi_file, "a") as f:
                f.write('\n'+strftime('%c')+'\n'+'x_start = '+str(x_start)+', dx = '+str(dx)+'\n'+'N_points = '+str(N_points)+'\n')
    if resume:
        # read last point in psi_file:
        with open(save_file, 'r') as f:
            psi_contents = f.read()
        ll = psi_contents.split('\n')[-2]
        pt = eval(ll.strip())
        x_start = pt[0] + dx
        # last_i = int(math.log(pt[0])*N_points/math.log(X))
        x_list = [x_start + i*dx for i in range(N_points)] 
        theta = pt[1]
        a = int(pt[0])
        L = []
        if verbose:
            print(f'Resuming psi computations from {save_file}; last point {pt}.')
        # now do the same for the pi_file:
        if pi_file is not None:
            with open(pi_file, 'r') as f:
                pi_contents = f.read()
            llp = pi_contents.split('\n')[-2]
            pt = eval(llp.strip())
            pi_val = pt[1]
            L_pi = []
            if verbose:
                print(f'Resuming *pi* computations from {save_file}; last point {pt}.')
    else: 
        x_list = [x_start + i*dx for i in range(N_points)]   # list of x-values at which to evaluate psi(x)
        theta = 0.0
        pi_val = 0
        a = 0 
        L = []
        L_pi = []
    for i, x in enumerate(x_list):
        b = int(x)
        # now the range in which to compute is (a+1, b+1)
        # If b-a is too big, split it up further
        ranges = [(i, min(i + MAX_INPUT_RANGE, b)) for i in range(a, b, MAX_INPUT_RANGE)]
        if verbose:
            print(f'x = {x}')
            if len(ranges) > 1:
                print(f" Splitting into {len(ranges)} blocks.")
            
        for j, (aa, bb) in enumerate(ranges):
            if verbose:
                print(f" - Block {j+1} of {len(ranges)}: {(aa, bb)}")
            if bb > aa:
                logging.info(f' - Sending block {j+1} of {len(ranges)}: ({aa}, {bb}) to GPU.')
                result = theta_pi_kernel(aa, bb, verbose=verbose)
                # (new_theta, new_pi) = theta_pi_kernel(aa, bb, verbose=verbose)
                theta += result[0]
                pi_val += result[1]
                logging.info(f' - - Result: {result}')
                # theta += random_theta_kernel(aa, bb, verbose=verbose)
                # theta += test_theta_kernel(aa, bb, verbose=verbose)

        a = int(x)
        L.append((x, theta))
        L_pi.append((x, pi_val))
        logging.info(f' - - - Appending point x = {x}, theta(x) = {theta}, pi(x) = {pi_val}')
        if verbose:
            print(f"{datetime.now()}: Point {i+1} of {len(x_list)}, x = {x}, theta(x) = {theta}, pi(x) = {pi_val}")
        if save_file:
            with open(save_file,"a") as f:
                f.write(str((x, theta))+'\n')
        if pi_file:
            with open(pi_file,"a") as f:
                f.write(str((x, pi_val))+'\n')
        # elif resume_file:
        #     with open(resume_file,"a") as f:
        #         f.write(str((x,theta))+'\n')
    return L, L_pi

In [11]:
# small example:
theta_list, pi_list = theta_GPU_linear(x_start=0, dx=100000, N_points=100, save_file='Pell-theta-test.txt', pi_file='Pell-pi-test.txt')

x = 0
2024-08-11 02:52:43.103500: Point 1 of 100, x = 0, theta(x) = 0.0, pi(x) = 0
x = 100000
 - Block 1 of 1: (0, 100000)
   Sieving primes in interval [1, 100001)...
    2399 primes found. Sending block [5, ... ,99989] of size 18.74 kB to GPU.
     Result: theta =  -887.5653633006716;  pi = 741
2024-08-11 02:52:43.109468: Point 2 of 100, x = 100000, theta(x) = -887.5653633006716, pi(x) = 741
x = 200000
 - Block 1 of 1: (100000, 200000)
   Sieving primes in interval [100001, 200001)...
    2112 primes found. Sending block [100069, ... ,199933] of size 16.5 kB to GPU.
     Result: theta =  -234.76439762608555;  pi = 691
2024-08-11 02:52:43.140850: Point 3 of 100, x = 200000, theta(x) = -1122.3297609267572, pi(x) = 1432
x = 300000
 - Block 1 of 1: (200000, 300000)
   Sieving primes in interval [200001, 300001)...
    2012 primes found. Sending block [200029, ... ,299941] of size 15.72 kB to GPU.
     Result: theta =  -1017.8586108385086;  pi = 616
2024-08-11 02:52:43.151873: Point 4 of 